In [1]:
%run ../ipynb_util_tars.py

In [2]:
%run ../ipynb_load_data_natural.py

{'SDG': Value(dtype='int64', id=None), 'ABSTRACT': Value(dtype='string', id=None), 'id': Value(dtype='string', id=None), 'sdg_desc_short': Value(dtype='string', id=None), 'sdg_desc_long': Value(dtype='string', id=None), '__index_level_0__': Value(dtype='int64', id=None)}
Example instance:	 {'SDG': 8, 'ABSTRACT': 'The scheme gives enterprises with business activity in Norway a tax credit on their R&D projects. The R&D content must be approved by the Research Council of Norway ex ante. In 2009, the cap on expenses per enterprise for intramural R&D projects increased to NOK 5.5 million (previously it was N0K 4 million), and NOK11 million (previously it was NOK 8 million) for projects conducted at an R&D institution.', 'id': None, 'sdg_desc_short': None, 'sdg_desc_long': None, '__index_level_0__': 492}
id2label: {0: '1', 1: '2', 2: '3', 3: '4', 4: '5', 5: '6', 6: '7', 7: '8', 8: '9', 9: '10', 10: '11', 11: '12', 12: '13', 13: '14', 14: '15', 15: '16', 16: '17'}
label2id: {'1': 0, '2': 1, '

In [ ]:
test_dataset = load_dataset(
    "csv", data_files={"test": str(DATA_DIR_PATH / "zo_up_test.csv")}
)
test_dataset = test_dataset.rename_columns({"sdg": "SDG", "abstract": "ABSTRACT"})

def convert_sdg_to_0indexed_int(d):
    d["SDG"] = int(d["SDG"]) - 1
    return d

test_dataset = test_dataset.map(convert_sdg_to_0indexed_int)

In [3]:
import torch
from transformers import AutoTokenizer, LlamaForSequenceClassification

LLAMA_PATH = CHECKPOINT_PATH + "/final/meta-llama/Meta-Llama-3-8B-ft-zo_up/checkpoint-2212"
llama_tokenizer = AutoTokenizer.from_pretrained(LLAMA_PATH)
llama_model = LlamaForSequenceClassification.from_pretrained(
    LLAMA_PATH,
    label2id=label2id,
    id2label=id2label,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
llama_model.eval()
llama_model.config.pad_token_id = llama_tokenizer.pad_token_id

encoded_dataset = dataset.map(preprocess_data(llama_tokenizer, max_length=1024, padding="longest"), batched=True, remove_columns=dataset["train"].column_names)
encoded_dataset.set_format("torch")

from sklearn.metrics import classification_report

# manual evaluation to show classifcation_report
true_labels = []
logits = []

for batch in encoded_dataset["test"]:
    batch = {k: v.to(llama_model.device).unsqueeze(0) for k, v in batch.items()}
    label = batch.pop("label")

    # Forward pass
    with torch.no_grad():
        out = llama_model(**batch)

    true_labels.append(label.item())
    logits.extend(out.logits.tolist())

probabilites = torch.nn.functional.softmax(torch.tensor(logits), dim=-1)
pred_labels = torch.argmax(probabilites, dim=-1).tolist()

report = classification_report(true_labels, pred_labels, target_names=[f"SDG {id2label[i]}" for i in range(len(labels))], digits=4)
print(report)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/dill/_dill.py:414: PicklingWarning: Cannot locate reference to <enum 'DatasetType'>.
  StockPickler.save(self, obj, save_persistent_id)
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/dill/_dill.py:414: PicklingWarning: Cannot pickle <enum 'DatasetType'>: __main__.DatasetType has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)


Map:   0%|          | 0/630 [00:00<?, ? examples/s]

Map:   0%|          | 0/271 [00:00<?, ? examples/s]

              precision    recall  f1-score   support

       SDG 1     0.7368    0.8235    0.7778        17
       SDG 2     0.9375    0.8824    0.9091        17
       SDG 3     0.7500    0.8824    0.8108        17
       SDG 4     1.0000    1.0000    1.0000        17
       SDG 5     0.7647    0.7647    0.7647        17
       SDG 6     0.9412    1.0000    0.9697        16
       SDG 7     0.8000    0.7500    0.7742        16
       SDG 8     0.7778    0.8235    0.8000        17
       SDG 9     0.7059    0.7059    0.7059        17
      SDG 10     0.5385    0.4118    0.4667        17
      SDG 11     0.8889    0.9412    0.9143        17
      SDG 12     0.9231    0.7059    0.8000        17
      SDG 13     0.7778    0.8235    0.8000        17
      SDG 14     1.0000    1.0000    1.0000        17
      SDG 15     0.8125    0.7647    0.7879        17
      SDG 16     0.6842    0.7647    0.7222        17
      SDG 17     0.0000    0.0000    0.0000         1

    accuracy              

In [4]:
from transformers import Trainer, TrainingArguments, EvalPrediction
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def compute_metrics(pred: EvalPrediction):
    labels = pred.label_ids
    accuracy = accuracy_score(labels, pred.predictions.argmax(-1))
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred.predictions.argmax(-1), average="weighted")
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

eval_trainer = Trainer(
    model=llama_model,
    args=TrainingArguments(output_dir="./eval_output", per_device_eval_batch_size=4),
    eval_dataset=encoded_dataset["test"],
    compute_metrics=compute_metrics
)
eval_results = eval_trainer.evaluate()
print(eval_results)

/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dvdblk (ngmi). Use `wandb login --relogin` to force relogin


{'eval_loss': 0.9594660401344299, 'eval_accuracy': 0.8081180811808119, 'eval_precision': 0.8060808665957582, 'eval_recall': 0.8081180811808119, 'eval_f1': 0.8050867543472591, 'eval_runtime': 40.6319, 'eval_samples_per_second': 6.67, 'eval_steps_per_second': 1.674}


In [5]:
pred_labels

[3,
 6,
 6,
 5,
 3,
 8,
 5,
 14,
 11,
 8,
 1,
 11,
 0,
 3,
 16,
 8,
 2,
 13,
 0,
 4,
 3,
 2,
 0,
 0,
 15,
 15,
 4,
 14,
 0,
 4,
 9,
 11,
 15,
 15,
 13,
 11,
 3,
 10,
 10,
 13,
 5,
 0,
 1,
 3,
 0,
 10,
 10,
 1,
 2,
 14,
 0,
 8,
 7,
 0,
 0,
 1,
 12,
 4,
 10,
 11,
 2,
 15,
 12,
 6,
 0,
 13,
 11,
 12,
 7,
 0,
 7,
 1,
 3,
 7,
 6,
 5,
 4,
 5,
 4,
 3,
 14,
 3,
 5,
 12,
 13,
 14,
 14,
 12,
 7,
 12,
 2,
 10,
 14,
 14,
 10,
 15,
 9,
 11,
 3,
 9,
 2,
 2,
 7,
 0,
 13,
 15,
 9,
 10,
 5,
 4,
 13,
 12,
 8,
 2,
 10,
 0,
 15,
 11,
 14,
 1,
 8,
 7,
 2,
 7,
 5,
 3,
 3,
 6,
 7,
 14,
 1,
 12,
 10,
 14,
 4,
 15,
 0,
 6,
 13,
 14,
 2,
 10,
 9,
 12,
 2,
 1,
 4,
 7,
 7,
 5,
 5,
 8,
 8,
 13,
 5,
 13,
 15,
 7,
 4,
 2,
 2,
 6,
 7,
 8,
 13,
 6,
 15,
 13,
 4,
 1,
 12,
 2,
 9,
 6,
 4,
 1,
 9,
 12,
 5,
 15,
 6,
 5,
 6,
 3,
 7,
 15,
 7,
 15,
 2,
 8,
 9,
 2,
 5,
 7,
 1,
 10,
 3,
 4,
 13,
 10,
 6,
 0,
 5,
 11,
 9,
 9,
 2,
 11,
 8,
 13,
 14,
 8,
 15,
 12,
 0,
 14,
 9,
 13,
 4,
 10,
 12,
 2,
 8,
 12,
 12,
 1,
 15,
 1,
 13

In [6]:
# load scibert
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# load the best model
MODEL_PATH = CHECKPOINT_PATH + "/final/allenai/scibert_scivocab_cased-zo_up/checkpoint-432/"

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    id2label=id2label,
    label2id=label2id
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, do_lower_case=False, skip_special_tokens=False)
model.eval()

encoded_dataset = dataset.map(preprocess_data(tokenizer, max_length=512, padding=True), batched=True, remove_columns=dataset["train"].column_names)
encoded_dataset.set_format("torch")

# manual evaluation to show classifcation_report
true_labels = []
logits = []

for batch in encoded_dataset["test"]:
    batch = {k: v.to(model.device).unsqueeze(0) for k, v in batch.items()}
    label = batch.pop("label")

    # Forward pass
    with torch.no_grad():
        out = model(**batch)

    true_labels.append(label.item())
    logits.extend(out.logits.tolist())

probabilites = torch.nn.functional.softmax(torch.tensor(logits), dim=-1)
pred_labels = torch.argmax(probabilites, dim=-1).tolist()

report = classification_report(true_labels, pred_labels, target_names=[f"SDG {id2label[i]}" for i in range(len(labels))], digits=4)
print(report)

/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/dill/_dill.py:414: PicklingWarning: Cannot locate reference to <enum 'DatasetType'>.
  StockPickler.save(self, obj, save_persistent_id)
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/dill/_dill.py:414: PicklingWarning: Cannot pickle <enum 'DatasetType'>: __main__.DatasetType has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)


Map:   0%|          | 0/630 [00:00<?, ? examples/s]

Map:   0%|          | 0/271 [00:00<?, ? examples/s]

              precision    recall  f1-score   support

       SDG 1     0.8235    0.8235    0.8235        17
       SDG 2     0.7727    1.0000    0.8718        17
       SDG 3     0.7368    0.8235    0.7778        17
       SDG 4     0.8889    0.9412    0.9143        17
       SDG 5     0.7857    0.6471    0.7097        17
       SDG 6     0.8421    1.0000    0.9143        16
       SDG 7     0.6316    0.7500    0.6857        16
       SDG 8     0.6875    0.6471    0.6667        17
       SDG 9     0.6250    0.5882    0.6061        17
      SDG 10     0.5333    0.4706    0.5000        17
      SDG 11     0.8824    0.8824    0.8824        17
      SDG 12     0.7273    0.4706    0.5714        17
      SDG 13     0.8000    0.7059    0.7500        17
      SDG 14     0.8889    0.9412    0.9143        17
      SDG 15     0.8571    0.7059    0.7742        17
      SDG 16     0.5714    0.7059    0.6316        17
      SDG 17     0.0000    0.0000    0.0000         1

    accuracy              

/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

In [7]:
print(pred_labels)

[3, 2, 6, 5, 3, 8, 5, 14, 15, 8, 1, 11, 0, 3, 14, 8, 2, 13, 7, 4, 0, 2, 0, 0, 15, 15, 7, 14, 0, 4, 9, 11, 15, 9, 13, 11, 3, 10, 10, 13, 5, 0, 1, 3, 0, 10, 10, 1, 4, 14, 9, 8, 7, 2, 0, 1, 11, 4, 10, 11, 1, 15, 12, 6, 0, 13, 11, 6, 9, 1, 7, 1, 3, 13, 6, 5, 4, 5, 15, 3, 5, 3, 5, 12, 13, 14, 14, 12, 7, 6, 2, 10, 14, 14, 6, 15, 9, 8, 3, 2, 2, 4, 6, 0, 13, 15, 11, 10, 5, 4, 13, 12, 8, 2, 10, 0, 15, 11, 14, 1, 8, 7, 2, 7, 5, 3, 3, 6, 7, 14, 1, 12, 10, 5, 2, 15, 0, 6, 13, 14, 2, 10, 3, 14, 13, 1, 4, 9, 7, 5, 5, 8, 3, 13, 5, 13, 15, 7, 4, 4, 2, 6, 7, 2, 13, 6, 9, 13, 4, 1, 12, 2, 0, 6, 0, 1, 9, 12, 5, 15, 12, 5, 6, 3, 7, 15, 7, 15, 2, 2, 9, 1, 5, 7, 1, 10, 3, 4, 13, 10, 9, 10, 5, 6, 9, 15, 2, 6, 8, 13, 14, 8, 15, 12, 0, 8, 9, 12, 4, 9, 6, 2, 15, 11, 12, 1, 15, 1, 13, 11, 13, 8, 10, 6, 8, 1, 8, 9, 0, 6, 4, 12, 11, 5, 14, 10, 12, 10, 1, 15, 12, 8, 9, 15, 1, 7, 8, 2, 7, 10, 1, 15, 6, 3, 3, 1, 3, 0, 1, 5, 12]
